In [1]:
from pathlib import Path
import numpy as np

INPUT_PATH = Path("input")

In [2]:
with open(INPUT_PATH / "day11_input.txt", "r") as f:
    array = [[c for c in l.rstrip()] for l in f.readlines()]

In [3]:
def transform_seat_pt1(array, row, col):
    adjacent_rows = [max(0, row - 1), min(row + 1, len(array) - 1)]
    adjacent_cols = [max(0, col - 1), min(col + 1, len(array[0]) - 1)]

    seat_status, adj_seat_status = array[row][col], []
    for r in range(adjacent_rows[0], adjacent_rows[1] + 1):
        for c in range(adjacent_cols[0], adjacent_cols[1] + 1):
            if array[r][c] != "." and (r, c) != (row, col):
                adj_seat_status.append(array[r][c])

    if seat_status == "L" and "#" not in adj_seat_status:
        return "#"
    elif seat_status == "#" and adj_seat_status.count("#") >= 4:
        return "L"
    elif seat_status == ".":
        return "."
    else:
        return seat_status


def update_seat_plan(array, transform_func):
    new_array = []
    for i, row in enumerate(array):
        new_row = []
        for j, col in enumerate(row):
            new_row.append(transform_func(array, i, j))
        new_array.append(new_row)
    return new_array


def musical_chairs(array, transform_func, verbose=False, use_ndarray=False):
    new_array, count = array, 0

    if use_ndarray:
        array = np.array(array)

    while True:
        new_array = update_seat_plan(array, transform_func)
        if use_ndarray:
            new_array = np.array(new_array)
        if not np.array_equal(new_array, array):
            array = new_array
            count += 1
            if verbose:
                print(count)
        else:
            break

    if use_ndarray:
        new_array = new_array.tolist()

    print(
        f"Stopped after {count} updates. {sum([x.count('#') for x in new_array])} occupied seats"
    )
    return new_array

In [4]:
out = musical_chairs(array, transform_seat_pt1)

Stopped after 102 updates. 2249 occupied seats


# Part 2

In [5]:
def check_diagonal(array, row, col, rotate=False):
    if rotate:
        row = len(array) - 1 - row
        diag = np.diagonal(np.rot90(array), offset=col - row)[::-1]
    else:
        diag = np.diagonal(array, offset=col - row)
    return check_both_sides(diag, min(row, col))


def check_both_sides(vector, seat_idx):
    left = "".join(vector[:seat_idx]).replace(".", "")
    right = "".join(vector[seat_idx + 1:]).replace(".", "")
    left_occupied = 1 if left.endswith("#") else 0
    right_occupied = 1 if right.startswith("#") else 0
    return left_occupied + right_occupied


def count_visible(array, row, col):
    visible = 0
    visible += check_both_sides(array[row], col)  # Check horizontally
    visible += check_both_sides(array[:, col], row)  # Check vertically
    visible += check_diagonal(array, row, col,
                              rotate=False)  # Check diagonally
    visible += check_diagonal(array, row, col,
                              rotate=True)  # Check diagonally (rotated)
    return visible


def transform_seat_pt2(array, row, col):
    seat_status = array[row, col]
    if seat_status == "L" and count_visible(array, row, col) == 0:
        return "#"
    elif seat_status == "#" and count_visible(array, row, col) >= 5:
        return "L"
    elif seat_status == ".":
        return "."
    else:
        return seat_status

In [6]:
with open(INPUT_PATH / "day11_input.txt", "r") as f:
    array = [[c for c in l.rstrip()] for l in f.readlines()]

In [7]:
out = musical_chairs(array, transform_seat_pt2, True, True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
Stopped after 72 updates. 2422 occupied seats


Seems to be a Heisenbug somewhere since the code produces the expected result for the example array...

Answer should be 2023 after 85 updates  :(

# Retry
This is a nice solution: https://github.com/davepage-mcr/aoc2020/blob/main/day11/seats.py

In [8]:
def count_visible_v2(array, row, col):
    neighbours = 0

    # Look for first seat in 8 cardinal directions from (col, row)
    for dc, dr in [(-1, -1), (0, -1), (1, -1), (-1, 0), (1, 0), (-1, +1),
                   (0, 1), (1, 1)]:

        occupied = False

        r = row + dr
        c = col + dc

        while r >= 0 and r < len(array) and c >= 0 and c < len(array[0]):
            if array[r][c] == '#':
                occupied = True
                break
            elif array[r][c] == 'L':
                break

            r += dr
            c += dc

        if occupied:
            neighbours += 1

    return (neighbours)


def transform_seat_pt2_v2(array, row, col):
    seat_status = array[row][col]
    if seat_status == "L" and count_visible_v2(array, row, col) == 0:
        return "#"
    elif seat_status == "#" and count_visible_v2(array, row, col) >= 5:
        return "L"
    elif seat_status == ".":
        return "."
    else:
        return seat_status

In [9]:
out = musical_chairs(array, transform_seat_pt2_v2)

Stopped after 85 updates. 2023 occupied seats
